# Hello Guys! 
* Hope you are doing well. This is my notebook where I have worked on predicting insurance cost based on the features provided in the data set
* I wanted to test the dataset with different regression models
* I have also done feature engineering initially and then exploratory analysis to build an understanding of the relationship between variables.
* Hope you Enjoy!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [ ]:
df = pd.read_csv('../input/insurance/insurance.csv')

In [ ]:
df.head()

# Data Cleaning and Feature Engineering 

In [ ]:
msno.matrix(df)

In [ ]:
df[df.isnull()].count()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
Male = pd.get_dummies(df['sex'], drop_first=True)
df = pd.concat([df, Male], axis=1 )

In [ ]:
Smoker = pd.get_dummies(df['smoker'], drop_first=True)
df = pd.concat([df, Smoker], axis=1 )

In [ ]:
df = df.rename(columns={'yes':'Smoker'})

In [ ]:
df['region'].unique()

In [ ]:
region = pd.get_dummies(df['region'])
df = pd.concat([df, region], axis=1 )
#df.drop('region', axis=1,inplace=True)
#df.drop(['sex','smoker'], axis=1, inplace=True)

In [ ]:
df.head()

* We will be keeping the categorical columns for now because we want to perform exploratory analysis on them
* Will drop them before applying predictive models

In [ ]:
plt.figure(figsize=(12,6))
sns.set_style('white')
sns.countplot(x='sex', data = df, palette='GnBu')
sns.despine(left=True)


In [ ]:
plt.figure(figsize=(14,10))
sns.set_style('white')
sns.boxplot(x='sex', y='charges', data = df, palette='OrRd', hue='Smoker')
sns.despine(left=True)


In [ ]:
fig, ax =plt.subplots(nrows= 1, ncols = 3, figsize= (14,6))
sns.scatterplot(x='age', y='charges', data = df, palette='coolwarm', hue='sex', ax=ax[0])
sns.scatterplot(x='age', y='charges', data = df, palette='GnBu', hue='Smoker', ax=ax[1])
sns.scatterplot(x='age', y='charges', data = df, palette='magma_r', hue='region', ax=ax[2])
sns.set_style('dark')
sns.despine(left=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
fig, ax =plt.subplots(nrows= 1, ncols = 2, figsize= (14,6))
sns.boxplot(x='region', y='charges', data = df, palette='GnBu', hue='Smoker', ax=ax[0])
sns.boxplot(x='region', y='charges', data = df, palette='coolwarm', hue='sex', ax=ax[1])

In [ ]:
fig, ax =plt.subplots(nrows= 1, ncols = 2, figsize= (14,6))
sns.scatterplot(x='bmi', y='charges', data = df, palette='GnBu_r', hue='sex', ax=ax[0])
sns.scatterplot(x='bmi', y='charges', data = df, palette='magma', hue='Smoker', ax=ax[1])
sns.set_style('dark')
sns.despine(left=True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
df.drop(['sex', 'region', 'smoker', 'southwest'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(16,6))
sns.heatmap(df.corr(), cmap='OrRd')

# Predictive Analysis

In [ ]:
X=df.drop('charges', axis=1)
y=df['charges']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_validate = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dense(units = 3, activation = 'relu'))
#model.add(Dropout(0.5))

#model.add(Dense(units = 2, activation = 'relu'))
#model.add(Dense(units = 4, activation = 'relu'))
#model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))



model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mse')
early_stop = EarlyStopping(monitor='val_loss', mode= 'min', verbose= 0, patience=15)


In [ ]:
model.fit(x=X_train, y=y_train, epochs = 2000, validation_data=(X_test, y_test), batch_size=128, callbacks=[early_stop])

In [ ]:
loss = pd.DataFrame(model.history.history)

In [ ]:
loss.plot()

In [ ]:
from sklearn.metrics import mean_squared_error
pred = model.predict(X_test)
np.sqrt(mean_squared_error(y_test,pred))

In [ ]:
entry_1 = df[:][257:477].drop('charges', axis=1)
pred = model.predict(entry_1)
np.sqrt(mean_squared_error(df[:][257:477]['charges'], pred))